In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import warnings
warnings.filterwarnings('ignore')

import os

In [2]:
brand_shoptype = pd.read_csv('/kaggle/input/ptr-rd1/Extra_material_2.csv')
transaction = pd.read_csv('/kaggle/input/ptr-rd1/Extra_material_3.csv', parse_dates=['date_id'])

print('brand_shoptype shape:', brand_shoptype.shape)
print('transaction shape:', transaction.shape)

brand_shoptype shape: (190754, 3)
transaction shape: (1048575, 6)


In [3]:
all_brands = np.sort(brand_shoptype.brand.unique())

## Task

Write a function(s) that takes in a brand as an input and answer the following question:

> ### The Top 3 itemids (in a list) from the ‘Official Shop’ of that particular brand that generated the highest Gross Sales Revenue from 10th May to 31st May 2019.

Note: Not all brands will have 3 itemids, in cases where there are none output should return N.A

In [4]:
# filter only official shop
brand_shoptype = brand_shoptype[brand_shoptype['shop_type'] == 'Official Shop']

brand_shoptype['shop_id'] = brand_shoptype['shop_id'].astype(int)

In [5]:
# filter only transaction from 10th May to 31st May
start_date = '2019-5-10'
end_date = '2019-5-31'

mask = (transaction['date_id'] >= start_date) & (transaction['date_id'] <= end_date)

transaction = transaction.loc[mask]

In [6]:
# drop duplicates row
transaction.drop_duplicates(inplace=True)
brand_shoptype.drop_duplicates(inplace=True)

* If you check, you'll found 'Anessa, Senka, Za, Tsubaki, Ma Cherie' in brand unique values
* I decided to drop this so that every shop_id only have 1 brand, not mix

In [7]:
error_idx = brand_shoptype[brand_shoptype['brand'] == 'Anessa, Senka, Za, Tsubaki, Ma Cherie'].index

brand_shoptype.drop(error_idx, inplace=True)

In [8]:
# filter only shop_id which is exist in transaction
brand_shoptype = brand_shoptype[brand_shoptype['shop_id'].isin(transaction['shopid'])]

In [9]:
brand_shoptype.head()

,shop_id,brand,shop_type
49,64903618,Harman Kardon,Official Shop
89,58542133,Sabina,Official Shop
102,75810249,Bosch,Official Shop
113,66829677,Nokia,Official Shop
131,82301700,The Body Shop,Official Shop


In [10]:
transaction.head()

,orderid,itemid,date_id,amount,item_price_usd,shopid
0,1316773501,1355135789,2019-05-16,1,5.483204,65948755
1,1316774684,1257902741,2019-05-16,1,0.387050,61364537
2,1316803330,2126770455,2019-05-16,1,0.344044,4980129
3,1317196434,2112999088,2019-05-16,1,6.704561,29089750
4,1318105348,1832131969,2019-05-16,1,5.354187,94372493


In [11]:
# there are 4 shop_id that have 2 brands
brand_shoptype.groupby('shop_id')['brand'].nunique().sort_values(ascending=False)[:5]

shop_id
64903618    2
61963112    2
75810249    2
35284879    2
52679373    1
Name: brand, dtype: int64

In [12]:
transaction.rename(columns={'shopid': 'shop_id'}, inplace=True)

final_df = brand_shoptype.merge(transaction, on='shop_id', how='left')

final_df.head()

,shop_id,brand,shop_type,orderid,itemid,date_id,amount,item_price_usd
0,64903618,Harman Kardon,Official Shop,1321879298,1921091854,2019-05-16,1,59.777675
1,58542133,Sabina,Official Shop,1321039647,1787516454,2019-05-16,1,7.246430
2,58542133,Sabina,Official Shop,1319191755,1618557460,2019-05-16,1,7.074408
3,58542133,Sabina,Official Shop,1375827952,1755241806,2019-05-29,1,9.203182
4,58542133,Sabina,Official Shop,1375901974,1648157592,2019-05-29,1,7.074408


In [13]:
final_df['gross_sales_revenue'] = final_df['amount'].values * final_df['item_price_usd'].values

In [14]:
final_df.sample(5)

,shop_id,brand,shop_type,orderid,itemid,date_id,amount,item_price_usd,gross_sales_revenue
3702,61792033,Durex,Official Shop,1377246974,1031991803,2019-05-29,1,1.247160,1.247160
6721,111139273,Yves Rocher,Official Shop,1368165371,1719453404,2019-05-27,1,12.875853,12.875853
7285,89960894,P&G,Official Shop,1360440330,2054825465,2019-05-25,1,4.472574,4.472574
8322,52679373,Logitech,Official Shop,1375433541,952669750,2019-05-29,1,2.537326,2.537326
5473,78546729,Johnson & Johnson,Official Shop,1333138765,1620690807,2019-05-19,1,4.257547,4.257547


## Task (look again for better understanding)

Write a function(s) that takes in a brand as an input and answer the following question:

> ### The Top 3 itemids (in a list) from the ‘Official Shop’ of that particular brand that generated the highest Gross Sales Revenue from 10th May to 31st May 2019.

Note: Not all brands will have 3 itemids, in cases where there are none output should return N.A

In [15]:
for_submission = pd.DataFrame({})

for_submission['Index'] = range(1, len(all_brands) + 1)
for_submission['Answers'] = ''

for idx, brand in enumerate(all_brands):
    result = final_df[final_df['brand'] == brand].groupby('itemid')['gross_sales_revenue'].sum().rename('gross_sales').sort_values(ascending=False)[:3].reset_index()
    if len(result['itemid']) > 0:
        for_submission.at[idx, 'Answers'] += (str(brand) + ', ')
        for itemid in result['itemid']:
            for_submission.at[idx, 'Answers'] += (str(itemid) + ', ')
    else:
        for_submission.at[idx, 'Answers'] += (str(brand) + ', N.A,,')
        
for_submission['Answers'] = for_submission['Answers'].apply(lambda x: x[:-2])

In [16]:
for_submission.to_csv('submission1.csv', index=False)

## Result:

* This submission only get 0.88888 in Public Score
* But got 0.97119 in Private Score